# Convolutional Neural Network

### Importing the libraries

In [1]:
!pip install --upgrade tensorflow keras Pillow

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import LeakyReLU

In [3]:
tf.__version__

'2.4.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [6]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=LeakyReLU(alpha="0.1"), input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=LeakyReLU(alpha="0.1")))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation=LeakyReLU(alpha="0.1")))

### Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 27s 108ms/step - loss: 0.6897 - accuracy: 0.5613 - val_loss: 0.6282 - val_accuracy: 0.6315
Epoch 2/25
250/250 [==============================] - 28s 114ms/step - loss: 0.5827 - accuracy: 0.6890 - val_loss: 0.6016 - val_accuracy: 0.6540
Epoch 3/25
250/250 [==============================] - 28s 114ms/step - loss: 0.5435 - accuracy: 0.7166 - val_loss: 0.5419 - val_accuracy: 0.7345
Epoch 4/25
250/250 [==============================] - 27s 110ms/step - loss: 0.5106 - accuracy: 0.7459 - val_loss: 0.5161 - val_accuracy: 0.7475
Epoch 5/25
250/250 [==============================] - 28s 114ms/step - loss: 0.4641 - accuracy: 0.7770 - val_loss: 0.5052 - val_accuracy: 0.7665
Epoch 6/25
250/250 [==============================] - 26s 104ms/step - loss: 0.4362 - accuracy: 0.7965 - val_loss: 0.4766 - val_accuracy: 0.7780
Epoch 7/25
250/250 [==============================] - 28s 112ms/step - loss: 0.4259 - accuracy: 0.8016 - val_loss: 0.4807 - val_ac

## Part 4 - Making a single prediction

In [15]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [16]:
print(prediction)

dog


## Fim